In [1]:
import numpy as np
import sys
import os
import time

In [3]:
import cv2 as cv

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

## Loading an image

In [ ]:
img_path = "../leaf-image-sequences/raw/2023/20230525/20230525_172420_ESWW0070020_1.JPG"
img = cv.imread(img_path) # loads image

if img is None:
    sys.exit("Could not read the image.")

cv.imshow("Display window", img) # display image
k = cv.waitKey(0) # wait until a key is pressed
 
if k == ord("s"): # check if pressed key is 's'
    cv.imwrite("test.png", img) # save image